In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sym
from scipy import constants as C

For galilean telescopes we have that the limit magnitude are:
$$ M = 12.5 + 5log_{10}(D) \quad \quad \text{D is the telescopes diameter}$$
And the area that can be reached are determinate for the type of telescope (Newtonian and galilean).

In [6]:
def TelescopeInfo(diameter,type):
    Ml = 12.5 + 5*np.log10(diameter)
    if type == 'newtonian':
        visualc = (diameter / 10) #in degrees
    elif type == 'galilean':
        visualc = (diameter / 15) 
    area = visualc ** 2
    return round(Ml,2),round(area,2)

ml,a = TelescopeInfo(18,'newtonian') 
print('For the given data the limit magnitude are {} and the covered area are {} degrees'.format(ml,a))

For the given data the limit magnitude are 18.78 and the covered area are 3.24 degrees
